In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('../backup/processed.csv', sep=';')
df_102 = df[df['mapped_veh_id'] == 102]

In [ ]:
data = df_102.copy()
# Sort by timestamps_UTC
timestamp_col = 'timestamps_UTC'
data[timestamp_col] = pd.to_datetime(data[timestamp_col])
data = data.sort_values(by=timestamp_col)
# # Remove data when consecutive timestamps delta is bigger than 30min
# data = data[data[timestamp_col].diff() < pd.Timedelta(minutes=30)]
# # Remove data when consecutive timestamps delta is smaller than 1s
# data = data[data[timestamp_col].diff() > pd.Timedelta(seconds=1)]

AirTemp = data['RS_E_InAirTemp_PC2']
Timestamps = data['timestamps_UTC']
limit = len(data)

ts = pd.Series(AirTemp.values, index=Timestamps)
ts = ts[0:limit]

# Normalize the data between 0 and 1
ts = (ts - ts.min()) / (ts.max() - ts.min())

In [ ]:
data_speed = data[['speed', 'timestamps_UTC']]

# Remove data when speed is 0
# data = data[data['speed'] != 0]

# Remove data when speed is bigger than 150km/h (shouldn't happen, max speed is 120km/h)
data_speed = data_speed[data_speed['speed'] <= 150 / 3.6]

# speed_ts = pd.Series(data_speed['speed'].values, index=data_speed['timestamps_UTC']).iloc[11250:11270]

# print(speed_ts)

speed_ts = pd.Series(data_speed['speed'].values, index=data_speed['timestamps_UTC'])

# Plot the data
plt.figure(figsize=(20,10))
plt.plot(speed_ts)

# Plot red dots where speed is above 120km/h (train max speed)
plt.plot(speed_ts[speed_ts > 120 / 3.6], 'ro') # Outliers because too fast
plt.plot(speed_ts[speed_ts <= 0.1], 'bo') # Movements due to GPS error

plt.show()

In [ ]:
df_sample = df_102.copy()

# Select the rows with the following timestamps
# 2023-02-02 07:11:06     20.774122
# 2023-02-02 07:11:13     14.818934
# 2023-02-02 07:12:06     10.395424
# 2023-02-02 07:12:13      6.529558
# 2023-02-02 07:16:13      8.051485
# 2023-02-02 07:17:09     18.215998
# 2023-02-02 07:17:13    255.023974
# 2023-02-02 07:18:09     36.929668
# 2023-02-02 07:18:14    413.612284
# 2023-02-02 07:19:09     38.416082
# 2023-02-02 07:19:14    422.576903
# 2023-02-02 07:20:09     51.021562
# 2023-02-02 07:21:08      0.000000
# 2023-02-02 07:22:07     73.940563
# 2023-02-02 07:23:08     82.126230
# 2023-02-02 07:24:08     90.935003
# 2023-02-02 07:25:08    111.698604
# 2023-02-02 07:26:11    123.009707
# 2023-02-02 07:27:11    138.893617
# 2023-02-02 07:28:12    139.785914

df_sample = df_sample[df_sample['timestamps_UTC'].isin([
    '2023-02-02 07:11:06',
    '2023-02-02 07:11:13',
    '2023-02-02 07:12:06',
    '2023-02-02 07:12:13',
    '2023-02-02 07:16:13',
    '2023-02-02 07:17:09',
    '2023-02-02 07:17:13',
    '2023-02-02 07:18:09',
    '2023-02-02 07:18:14',
    '2023-02-02 07:19:09',
    '2023-02-02 07:19:14',
    '2023-02-02 07:20:09',
    '2023-02-02 07:21:08',
    '2023-02-02 07:22:07',
    '2023-02-02 07:23:08',
    '2023-02-02 07:24:08',
    '2023-02-02 07:25:08',
    '2023-02-02 07:26:11',
    '2023-02-02 07:27:11',
    '2023-02-02 07:28:12'
])]

df_sample

In [ ]:
# Plot the data
plt.figure(figsize=(20,10))
plt.plot(ts)
plt.show()

# Count the number of 0s in 'RS_E_InAirTemp_PC2'
print("Number of 0s in 'RS_E_InAirTemp_PC2':", len(data[data['RS_E_InAirTemp_PC2'] == 0]))

# Remove data when 'RS_E_InAirTemp_PC2' is 0
data_clean = data[data['RS_E_InAirTemp_PC2'] != 0]
ts_clean = pd.Series(data_clean['RS_E_InAirTemp_PC2'].values, index=data_clean['timestamps_UTC'])
print("Size of the data after removing 0s in 'RS_E_InAirTemp_PC2':", len(ts_clean))

# Plot the data
plt.figure(figsize=(20,10))
plt.plot(ts_clean)
plt.show()

# Remove the rows where the distance between consecutive location is smaller than 1m
# + pay attention to rows where timestamp is the same or really close => use speed
# + pay attention to rows where speed is 0
# + pay attention to rows where speed is bigger than 100km/h (shouldn't happen, max speed is 100km/h)
print("Number of rows where speed smaller than 1km/h:", len(data_clean[data_clean['speed'] < 1]))
data_clean = data_clean[data_clean['speed'] >= 1 / 3.6]
print("Number of rows where speed bigger than 100km/h:", len(data_clean[data_clean['speed'] > 100]))
data_clean = data_clean[data_clean['speed'] <= 120 / 3.6]

# Transform the time_difference to time
data_clean['time_difference'] = pd.to_timedelta(data['time_difference'])
print("Number of rows where time difference is bigger than 10s:", len(data_clean[data_clean['time_difference'] > pd.Timedelta(seconds=10)]))
data_clean = data_clean[data_clean['time_difference'] < pd.Timedelta(seconds=10)]

ts_clean = pd.Series(data_clean['RS_E_InAirTemp_PC2'].values, index=data_clean['timestamps_UTC'])
print("Size of the data after removing rows where speed smaller than 1km/h and bigger than 100km/h:", len(ts_clean))

# Plot the data
plt.figure(figsize=(20,10))
plt.plot(ts_clean)
plt.show()


# Plot lines for each index, showcase real data portions
# timestamps = data_clean['timestamps_UTC']
# for i in tqdm(timestamps):
#     plt.axvline(x=i, color='r')
# plt.plot(ts_clean)
# plt.show()

In [ ]:
# Plot the data
plt.figure(figsize=(20,5))
plt.plot(ts)
plt.show()

# Separate data by duration without data
data_sample = data_clean.copy()

# Reset the index
data_sample = data_sample.reset_index(drop=True)

# Get the difference between consecutive timestamps
diff = data_sample['timestamps_UTC'].diff()

# Get the right side indexes where the difference is bigger than 30min
idx_left = diff[diff > pd.Timedelta(minutes=10)].index


# Get the left side indexes where the difference is bigger than 30min
idx_right = idx_left - 1

# Add the first and last indexes
idx_left = [0] + idx_left.tolist()
idx_right = idx_right.tolist() + [len(data_sample)]

# Plot the data outside the segments
ts = pd.Series(data_sample['RS_E_InAirTemp_PC2'].values, index=data_sample['timestamps_UTC'])

plt.figure(figsize=(20,5))
# for i in tqdm(data_sample['timestamps_UTC']):
#     plt.axvline(x=i, color='r')

plt.plot(ts)
plt.show()

plt.figure(figsize=(20,5))

for segment in zip(idx_left, idx_right):
    data_plot = 0
    ts_plot = ts[segment[0]:segment[1]]
    plt.plot(ts_plot, label=str(segment))

plt.savefig('segments.png', dpi=300)
plt.show()